# Libraries

In [1]:
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
# def read_files (path):
#     df = pd.read_csv (path) [['Path', 'ClassId']]
#     unique_classes = df ['ClassId'].unique ()
#     selected_classes = np.random.choice (unique_classes, size=8, replace=False)

#     selected_data = pd.DataFrame ()
#     for class_id in selected_classes:
#         class_images = df [df ['ClassId'] == class_id].sample (n=100)
#         selected_data = pd.concat ([selected_data, class_images], ignore_index=True)

#     selected_data = selected_data.sample(frac=1, random_state=42).reset_index(drop=True)
#     selected_data = selected_data.sort_values(by=['ClassId', 'Path']).reset_index(drop=True)

#     return selected_data

# read_files ("Train.csv").to_csv ("train_use.csv", index=False)

# Clip Function

In [3]:
def clip (image):
    image [image < 0] = 0
    image [image > 255] = 255

    return image.astype (np.uint8)

# Preprocessing and Filtering

In [4]:
def mean_filter (image):
    image = image.astype (np.float32)
    padded = np.pad (image, ((1,1),(1,1)), mode = 'edge')
    output = np.zeros_like (image)

    for i in range (image.shape [0]):
        for j in range (image.shape [1]):
            region = padded [i:i+3, j:j+3]
            output [i, j] = np.mean (region)

    return clip (output).astype (np.uint8)

In [5]:
def gaussian_kernel ():
    sigma = 1
    size = 3

    ax = np.linspace (-(size // 2), size // 2, size)
    xx, yy = np.meshgrid (ax, ax)
    kernel = np.exp (-(xx ** 2 + yy ** 2) / (2 * sigma ** 2))
    kernel /= (2 * np.pi * sigma ** 2)
    kernel /= np.sum (kernel)
    return kernel

def gaussian_filter (image):
    image = image.astype (np.float32)
    kernel = gaussian_kernel ()

    a, b = image.shape
    output = np.zeros_like (image)

    for i in range (a):
        for j in range (b):
            sum = 0
            for k in range (3):
                for l in range (3):
                    x = i + k - 1
                    y = j + l - 1

                    if 0 <= x < a and 0 <= y < b:
                        sum += image [x, y] * kernel [k, l]
            output [i, j] = sum

    return clip (output).astype (np.uint8)

In [6]:
def median_filter (image):
    image = image.astype (np.float32)
    padded = np.pad (image, ((1,1), (1,1)), mode='edge')
    output = np.zeros_like (image)

    for i in range (image.shape [0]):
        for j in range (image.shape [1]):
            region = padded [i : i+3, j : j+3]
            output [i, j] = np.median (region)

    return clip (output).astype (np.uint8)

In [7]:
def adaptive_median_filter (image, max_size=7):
    image = image.astype (np.float32)
    output = image.copy ()
    pad = max_size // 2
    padded = np.pad (image, pad, mode='edge')

    for i in range (image.shape[0]):
        for j in range (image.shape[1]):
            window_size = 3
            while window_size <= max_size:
                half = window_size // 2
                region = padded [i + pad - half : i + pad + half + 1,
                                j + pad - half : j + pad + half + 1]
                z_min = np.min (region)
                z_max = np.max (region)
                z_med = np.median (region)
                z_xy = padded [i + pad, j + pad]

                A1 = z_med - z_min
                A2 = z_med - z_max

                if A1 > 0 and A2 < 0:
                    B1 = z_xy - z_min
                    B2 = z_xy - z_max
                    if B1 > 0 and B2 < 0:
                        output [i, j] = z_xy
                    else:
                        output [i, j] = z_med
                    break
                else:
                    window_size += 2

            if window_size > max_size:
                output [i, j] = z_med

    return clip (output).astype (np.uint8)


In [8]:
def high_boost_filter (image, A=1.5):
    image = image.astype (np.float32)
    blurred = gaussian_filter (image)
    mask = image - blurred
    output = (A - 1) * image + mask
    return clip (output).astype (np.uint8)

# Color Space Conversion and Segmentation

In [9]:
# def rgb_to_hsv (image):
#     image = image.astype (np.float32) / 255.0
#     r, g, b = image [..., 0], image [..., 1], image [..., 2]
#     c_max = np.max (image, axis=2)
#     c_min = np.min (image, axis=2)
#     delta = c_max - c_min

#     h = np.zeros_like (c_max)
#     s = np.zeros_like (c_max)
#     v = c_max

#     mask = delta != 0
#     r_mask = (c_max == r) & mask
#     g_mask = (c_max == g) & mask
#     b_mask = (c_max == b) & mask

#     h [r_mask] = (60 * (g [r_mask] - b [r_mask]) / delta [r_mask]) % 360
#     h [g_mask] = (60 * (b [g_mask] - r [g_mask]) / delta [g_mask]) + 120
#     h [b_mask] = (60 * (r [b_mask] - g [b_mask]) / delta [b_mask]) + 240

#     h [~mask] = 0
#     s [c_max != 0] = delta [c_max != 0] / c_max [c_max != 0]
#     s [c_max == 0] = 0

#     hsv = np.stack ([h, s, v], axis=2)
#     return hsv

def rgb_to_hsv(image):
    image = image.astype(np.float32) / 255.0
    r, g, b = image[..., 0], image[..., 1], image[..., 2]
    c_max = np.max(image, axis=2)
    c_min = np.min(image, axis=2)
    delta = c_max - c_min

    h = np.zeros_like(c_max)
    s = np.zeros_like(c_max)
    v = c_max

    mask = delta != 0
    r_mask = (c_max == r) & mask
    g_mask = (c_max == g) & mask
    b_mask = (c_max == b) & mask

    h[r_mask] = (60 * (g[r_mask] - b[r_mask]) / delta[r_mask]) % 360
    h[g_mask] = (60 * (b[g_mask] - r[g_mask]) / delta[g_mask]) + 120
    h[b_mask] = (60 * (r[b_mask] - g[b_mask]) / delta[b_mask]) + 240

    h[~mask] = 0
    s[c_max != 0] = delta[c_max != 0] / c_max[c_max != 0]
    s[c_max == 0] = 0

    # Convert to OpenCV-style HSV format
    h = (h / 2).clip(0, 180)           # Hue ∈ [0,180]
    s = (s * 255).clip(0, 255)         # Saturation ∈ [0,255]
    v = (v * 255).clip(0, 255)         # Value ∈ [0,255]

    hsv = np.stack([h, s, v], axis=2).astype(np.uint8)
    return hsv

In [10]:
def segment_red_blue (hsv):
    h, s, v = hsv [..., 0], hsv [..., 1], hsv [..., 2]

    red_mask1 = (h >= 0) & (h <= 15) & (s >= 100) & (v >= 80)
    red_mask2 = (h >= 165) & (h <= 180) & (s >= 100) & (v >= 80)
    red_mask = red_mask1 | red_mask2

    blue_mask = (h >= 100) & (h <= 130) & (s >= 100) & (v >= 80)

    mask = red_mask | blue_mask
    return (mask * 255).astype (np.uint8)

# def segment_red_blue(hsv):
#     h, s, v = hsv[..., 0], hsv[..., 1], hsv[..., 2]

#     # Normalize H from [0, 360] if needed (OpenCV uses 0–180)
#     # Adjusted thresholds:
#     # Hue: [0–10] or [170–180] for RED
#     # Hue: [100–130] for BLUE
#     # Sat/Val thresholds increased for more vivid colors
    
#     red_mask1 = (h >= 0) & (h <= 10) & (s >= 0.5) & (v >= 0.5)
#     red_mask2 = (h >= 170) & (h <= 180) & (s >= 0.5) & (v >= 0.5)
#     red_mask = red_mask1 | red_mask2

#     blue_mask = (h >= 100) & (h <= 130) & (s >= 0.5) & (v >= 0.5)

#     # Combine red and blue masks
#     mask = red_mask | blue_mask

#     return (mask * 255).astype(np.uint8)

In [11]:

# def rgb_to_hsv(rgb):
#     rgb = rgb.astype(float) / 255.0
#     hsv = np.zeros_like(rgb)
    
#     r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
#     maxc = np.maximum(np.maximum(r, g), b)
#     minc = np.minimum(np.minimum(r, g), b)
    
#     hsv[:,:,2] = maxc
#     mask = maxc != 0
#     hsv[mask, 1] = (maxc - minc)[mask] / maxc[mask]
    
#     rc = np.zeros_like(r)
#     gc = np.zeros_like(g)
#     bc = np.zeros_like(b)
    
#     mask = maxc != minc
#     rc[mask] = (maxc - r)[mask] / (maxc - minc)[mask]
#     gc[mask] = (maxc - g)[mask] / (maxc - minc)[mask]
#     bc[mask] = (maxc - b)[mask] / (maxc - minc)[mask]
    
#     h = np.zeros_like(r)
    
#     mask = (maxc == r) & (maxc != minc)
#     h[mask] = bc[mask] - gc[mask]
    
#     mask = (maxc == g) & (maxc != minc)
#     h[mask] = 2.0 + rc[mask] - bc[mask]
    
#     mask = (maxc == b) & (maxc != minc)
#     h[mask] = 4.0 + gc[mask] - rc[mask]
    
#     h = h / 6.0
#     mask = h < 0
#     h[mask] += 1.0
    
#     hsv[:,:,0] = h * 180
#     hsv[:,:,1] *= 255
#     hsv[:,:,2] *= 255
    
#     return hsv.astype(np.uint8)

# def segment_red_blue(hsv_img):
#     h, s, v = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]
    
#     red_mask_low = (h <= 15) & (s >= 100) & (v >= 80)
#     red_mask_high = (h >= 165) & (h <= 180) & (s >= 100) & (v >= 80)
#     red_mask = red_mask_low | red_mask_high
    
#     blue_mask = (h >= 100) & (h <= 130) & (s >= 100) & (v >= 80)
    
#     mask = red_mask | blue_mask
    
#     if np.sum(red_mask) > np.sum(blue_mask):
#         color = 'red'
#     else:
#         color = 'blue'
    
#     return mask


In [12]:
def binary_threshold (image, threshold):
    binary = np.zeros_like (image, dtype=np.uint8)
    binary [image > threshold] = 255
    return binary

In [13]:
def erode (image, kernel_size=3):
    pad = kernel_size // 2
    padded = np.pad (image, pad, mode='edge')
    output = np.zeros_like (image)

    for i in range (image.shape [0]):
        for j in range (image.shape [1]):
            region = padded [i:i+kernel_size, j:j+kernel_size]
            output [i, j] = np.min (region)
    return output

def dilate (image, kernel_size=3):
    pad = kernel_size // 2
    padded = np.pad (image, pad, mode='edge')
    output = np.zeros_like (image)

    for i in range (image.shape [0]):
        for j in range (image.shape [1]):
            region = padded [i:i+kernel_size, j:j+kernel_size]
            output [i, j] = np.max (region)
    return output

def opening (image, kernel_size=3):
    return dilate (erode (image, kernel_size), kernel_size)


In [14]:
def connected_component_filtering (binary_image, area_threshold):
    h, w = binary_image.shape
    labels = np.zeros_like (binary_image, dtype=np.int32)
    output = np.zeros_like (binary_image)
    label = 1

    for i in range (h):
        for j in range (w):
            if binary_image [i, j] == 255 and labels [i, j] == 0:
                stack = [(i, j)]
                component = []

                while stack:
                    x, y = stack.pop ()
                    if (0 <= x < h) and (0 <= y < w):
                        if binary_image [x, y] == 255 and labels [x, y] == 0:
                            labels [x, y] = label
                            component.append ((x, y))
                            stack.extend ([(x+1,y), (x-1,y), (x,y+1), (x,y-1)])

                if len (component) >= area_threshold:
                    for (x, y) in component:
                        output [x, y] = 255

                label += 1

    return output


In [15]:
def fill_holes (binary_image):
    h, w = binary_image.shape
    visited = np.zeros_like (binary_image, dtype=bool)
    filled = binary_image.copy ()

    stack = []
    for i in range (h):
        if filled [i, 0] == 0:
            stack.append ((i, 0))
        if filled [i, w - 1] == 0:
            stack.append ((i, w - 1))
    for j in range (w):
        if filled [0, j] == 0:
            stack.append ((0, j))
        if filled [h - 1, j] == 0:
            stack.append ((h - 1, j))

    while stack:
        x, y = stack.pop ()
        if 0 <= x < h and 0 <= y < w:
            if filled [x, y] == 0 and not visited[x, y]:
                visited [x, y] = True
                stack.extend ([(x+1,y), (x-1,y), (x,y+1), (x,y-1)])

    for i in range (h):
        for j in range (w):
            if not visited [i, j] and filled [i, j] == 0:
                filled [i, j] = 255

    return filled


# Edge Detection

In [16]:
def sobel_gradients (image):

    Kx = np.array ([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], dtype=np.float32)
    Ky = np.array ([[1, 2, 1],
                   [0, 0, 0],
                   [-1, -2, -1]], dtype=np.float32)

    pad = 1
    padded = np.pad (image, pad, mode='reflect')
    Gx = np.zeros_like (image, dtype=np.float32)
    Gy = np.zeros_like (image, dtype=np.float32)

    for i in range (image.shape[0]):
        for j in range (image.shape[1]):
            region = padded [i:i+3, j:j+3]
            Gx [i, j] = np.sum (region * Kx)
            Gy [i, j] = np.sum (region * Ky)

    magnitude = np.sqrt (Gx**2 + Gy**2)
    direction = np.arctan2 (Gy, Gx)
    
    return magnitude, direction

In [17]:
def non_max_suppression (magnitude, direction):
    h, w = magnitude.shape
    output = np.zeros ((h, w), dtype=np.float32)
    angle = np.rad2deg (direction) % 180

    for i in range (1, h-1):
        for j in range (1, w-1):
            q, r = 255, 255

            if (0 <= angle [i,j] < 22.5) or (157.5 <= angle [i,j] <= 180):
                q = magnitude [i, j+1]
                r = magnitude [i, j-1]

            elif 22.5 <= angle [i,j] < 67.5:
                q = magnitude [i+1, j-1]
                r = magnitude [i-1, j+1]
            
            elif 67.5 <= angle [i,j] < 112.5:
                q = magnitude [i+1, j]
                r = magnitude [i-1, j]
            
            elif 112.5 <= angle [i,j] < 157.5:
                q = magnitude [i-1, j-1]
                r = magnitude [i+1, j+1]

            if magnitude [i,j] >= q and magnitude [i,j] >= r:
                output [i,j] = magnitude [i,j]
            else:
                output [i,j] = 0

    return output

In [18]:
def double_thresholding (img, low_thresh_ratio=0.05, high_thresh_ratio=0.15):
    high_thresh = img.max () * high_thresh_ratio
    low_thresh = high_thresh * low_thresh_ratio

    strong = 255
    weak = 75

    result = np.zeros_like (img, dtype=np.uint8)

    strong_i, strong_j = np.where (img >= high_thresh)
    weak_i, weak_j = np.where ((img <= high_thresh) & (img >= low_thresh))

    result [strong_i, strong_j] = strong
    result [weak_i, weak_j] = weak

    return result, weak, strong

In [19]:
def edge_tracking_by_hysteresis (img, weak, strong=255):
    h, w = img.shape
    for i in range (1, h-1):
        for j in range (1, w-1):
            if img [i,j] == weak:
                if ((img [i+1, j-1] == strong) or (img [i+1, j] == strong) or (img [i+1, j+1] == strong)
                    or (img [i, j-1] == strong) or (img [i, j+1] == strong)
                    or (img [i-1, j-1] == strong) or (img [i-1, j] == strong) or (img [i-1, j+1] == strong)):
                    img [i, j] = strong
                else:
                    img [i, j] = 0
    return img

# Geometric Normalization

In [20]:
def rotate_image (image, angle_deg):
    angle_rad = np.deg2rad (angle_deg)
    h, w = image.shape [:2]
    center = np.array ([w / 2, h / 2])

    R = np.array ([
        [np.cos (angle_rad), -np.sin (angle_rad)],
        [np.sin (angle_rad),  np.cos (angle_rad)]
    ])

    coords_y, coords_x = np.meshgrid (np.arange (h), np.arange (w), indexing='ij')
    coords = np.stack ([coords_x.ravel (), coords_y.ravel ()], axis=1)

    shifted = coords - center
    rotated = shifted @ R.T + center
    rotated = rotated.round ().astype (int)

    output = np.zeros_like (image)
    for i, (x_new, y_new) in enumerate (rotated):
        x_old, y_old = coords [i]
        if 0 <= y_new < h and 0 <= x_new < w:
            output [y_new, x_new] = image [y_old, x_old]
    return output

In [21]:
def scale_image (image, size=(200, 200)):
    h, w = image.shape [:2]
    new_h, new_w = size
    scale_y, scale_x = h / new_h, w / new_w

    y_indices = (np.arange (new_h) * scale_y).astype (int)
    x_indices = (np.arange (new_w) * scale_x).astype (int)
    
    return image [y_indices [:, None], x_indices]

In [22]:
def perspective_transform_numpy (image, src_pts, dst_pts, output_size=(200, 200)):
    
    def compute_homography (src, dst):
        A = []
        for (x, y), (u, v) in zip (src, dst):
            A.append ([-x, -y, -1, 0, 0, 0, x*u, y*u, u])
            A.append ([0, 0, 0, -x, -y, -1, x*v, y*v, v])
        A = np.array (A)
        _, _, Vt = np.linalg.svd (A)
        H = Vt [-1].reshape ((3, 3))
        return H / H [-1, -1]

    src = np.array (src_pts, dtype=np.float32)
    dst = np.array (dst_pts, dtype=np.float32)
    H = compute_homography (src, dst)

    output_h, output_w = output_size
    output = np.zeros ((output_h, output_w), dtype=image.dtype)

    for i in range (output_h):
        for j in range (output_w):
            target_coords = np.array ([j, i, 1])
            source_coords = np.linalg.inv (H) @ target_coords
            source_coords /= source_coords [2]
            x, y = int (round (source_coords [0])), int (round (source_coords [1]))
            if 0 <= y < image.shape [0] and 0 <= x < image.shape [1]:
                output [i, j] = image [y, x]

    return output

# Feature Extraction

In [23]:
# def convolve2d (image, kernel):
#     kh, kw = kernel.shape
#     pad_h, pad_w = kh // 2, kw // 2
#     padded = np.pad (image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')
#     output = np.zeros_like (image, dtype=float)

#     for i in range (image.shape [0]):
#         for j in range (image.shape [1]):
#             region = padded [i:i+kh, j:j+kw]
#             output [i, j] = np.sum (region * kernel)
#     return output

# def harris_corner_count (gray, k=0.04, threshold_ratio=0.01):
#     sobel_x = np.array ([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
#     sobel_y = np.array ([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

#     Ix = convolve2d (gray, sobel_x)
#     Iy = convolve2d (gray, sobel_y)

#     Ixx = Ix * Ix
#     Iyy = Iy * Iy
#     Ixy = Ix * Iy

#     kernel = np.array ([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16

#     Sxx = convolve2d (Ixx, kernel)
#     Syy = convolve2d (Iyy, kernel)
#     Sxy = convolve2d (Ixy, kernel)

#     detM = Sxx * Syy - Sxy ** 2
#     traceM = Sxx + Syy
#     R = detM - k * (traceM ** 2)

#     threshold = threshold_ratio * np.max (R)
#     corners = (R > threshold)
#     return np.sum (corners)

# def compute_perimeter (binary_mask):
#     eroded = np.zeros_like (binary_mask)
#     for i in range (1, binary_mask.shape [0]-1):
#         for j in range (1, binary_mask.shape[1]-1):
#             if binary_mask [i, j] == 1 and np.all (binary_mask [i-1:i+2, j-1:j+2] == 1):
#                 eroded [i, j] = 1
#     perimeter = np.sum (binary_mask) - np.sum (eroded)
#     return perimeter

# def extract_features (normalized_rgb, binary_mask, normalized_hsv):
#     gray = np.mean (normalized_rgb, axis=2).astype (np.float32)

#     # 1. Corner Count
#     corners = harris_corner_count (gray)

#     # 2. Circularity
#     area = np.sum (binary_mask)
#     perimeter = compute_perimeter (binary_mask)
#     circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter != 0 else 0

#     # 3. Bounding box and Aspect Ratio
#     ys, xs = np.where (binary_mask == 1)
#     if len (xs) == 0 or len (ys) == 0:
#         aspect_ratio = 0
#         extent = 0
#     else:
#         width = xs.max () - xs.min () + 1
#         height = ys.max () - ys.min () + 1
#         aspect_ratio = width / height if height != 0 else 0

#         # 4. Extent
#         bbox_area = width * height
#         extent = area / bbox_area if bbox_area != 0 else 0

#     # 5. Average Hue
#     hue = normalized_hsv [..., 0]
#     avg_hue = np.mean (hue [binary_mask == 1]) if np.any (binary_mask == 1) else 0

#     return {
#         "corner_count": int (corners),
#         "circularity": float (circularity),
#         "aspect_ratio": float (aspect_ratio),
#         "extent": float (extent),
#         "average_hue": float (avg_hue)
#     }


In [24]:
def convolve2d(image, kernel):
    """
    Perform 2D convolution of image with kernel
    """
    kh, kw = kernel.shape
    pad_h, pad_w = kh // 2, kw // 2
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')
    output = np.zeros_like(image, dtype=float)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded[i:i+kh, j:j+kw]
            output[i, j] = np.sum(region * kernel)
    return output

def harris_corner_count(gray, k=0.04, threshold_ratio=0.01, window_size=3):
    """
    Detect corners using Harris corner detection and return the count
    
    Args:
        gray: Grayscale image
        k: Harris detector free parameter (typically 0.04-0.06)
        threshold_ratio: Threshold as a fraction of maximum response
        window_size: Size of the window for gradient covariance calculation
        
    Returns:
        Number of detected corners
    """
    # Convert to float32 if not already
    gray = gray.astype(np.float32)
    
    # Compute gradients
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    Ix = convolve2d(gray, sobel_x)
    Iy = convolve2d(gray, sobel_y)

    # Compute products of derivatives
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    # Generate Gaussian kernel for weighted summation
    if window_size == 3:
        kernel = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16
    else:
        # Create a Gaussian kernel based on window_size
        sigma = 0.3 * ((window_size - 1) * 0.5 - 1) + 0.8
        x = np.arange(-(window_size // 2), window_size // 2 + 1)
        y = np.arange(-(window_size // 2), window_size // 2 + 1)
        X, Y = np.meshgrid(x, y)
        kernel = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
        kernel = kernel / np.sum(kernel)

    # Compute weighted sums
    Sxx = convolve2d(Ixx, kernel)
    Syy = convolve2d(Iyy, kernel)
    Sxy = convolve2d(Ixy, kernel)

    # Compute corner response
    detM = Sxx * Syy - Sxy**2
    traceM = Sxx + Syy
    R = detM - k * (traceM**2)

    # Threshold and count corners
    threshold = threshold_ratio * np.max(R)
    corners = (R > threshold)
    return np.sum(corners)

def compute_perimeter(binary_mask):
    """
    Compute perimeter of shapes in binary mask
    
    Args:
        binary_mask: Binary image (0 for background, non-zero for foreground)
        
    Returns:
        Perimeter length
    """
    # Ensure binary_mask is binary (0 and 1)
    binary = (binary_mask > 0).astype(np.uint8)
    
    # A pixel is on the boundary if it's 1 and has at least one 0 neighbor
    kernel = np.ones((3, 3), dtype=np.uint8)
    kernel[1, 1] = 0  # Center element is 0
    
    # Dilate and subtract to find boundary
    dilated = np.zeros_like(binary)
    for i in range(1, binary.shape[0]-1):
        for j in range(1, binary.shape[1]-1):
            if np.any(binary[i-1:i+2, j-1:j+2] * kernel > 0):
                dilated[i, j] = 1
    
    boundary = np.logical_and(dilated, binary)
    return np.sum(boundary)

def extract_features(normalized_rgb, binary_mask, normalized_hsv):
    """
    Extract features from processed image
    
    Args:
        normalized_rgb: RGB image normalized to standard size
        binary_mask: Binary mask of the segmented sign
        normalized_hsv: HSV version of the normalized image
        
    Returns:
        Dictionary of extracted features
    """
    # Convert to proper format
    if normalized_rgb.dtype != np.float32:
        gray = np.mean(normalized_rgb, axis=2).astype(np.float32)
    else:
        gray = np.mean(normalized_rgb, axis=2)
    
    # Ensure binary_mask is binary (0 and 1)
    binary = (binary_mask > 0).astype(np.uint8)
    
    # 1. Corner Count
    corners = harris_corner_count(gray)
    
    # 2. Circularity
    area = np.sum(binary)
    perimeter = compute_perimeter(binary)
    circularity = (4 * np.pi * area) / (perimeter**2) if perimeter > 0 else 0
    
    # 3. Bounding box and Aspect Ratio
    ys, xs = np.where(binary == 1)
    if len(xs) == 0 or len(ys) == 0:
        aspect_ratio = 0
        extent = 0
    else:
        x_min, x_max = xs.min(), xs.max()
        y_min, y_max = ys.min(), ys.max()
        width = x_max - x_min + 1
        height = y_max - y_min + 1
        aspect_ratio = width / height if height > 0 else 0
        
        # 4. Extent
        bbox_area = width * height
        extent = area / bbox_area if bbox_area > 0 else 0
    
    # 5. Average Hue
    # Handle different possible HSV formats
    if len(normalized_hsv.shape) == 3 and normalized_hsv.shape[2] >= 3:
        hue = normalized_hsv[:, :, 0]  # Standard HSV format
    elif len(normalized_hsv.shape) == 3 and normalized_hsv.shape[0] == 3:
        hue = normalized_hsv[0, :, :]  # Channel-first format
    else:
        hue = normalized_hsv  # Assume it's already the hue channel
    
    # Calculate average hue where mask is active
    if np.any(binary):
        avg_hue = np.mean(hue[binary == 1])
    else:
        avg_hue = 0
    
    return {
        "corner_count": int(corners),
        "circularity": float(circularity),
        "aspect_ratio": float(aspect_ratio),
        "extent": float(extent),
        "average_hue": float(avg_hue)
    }

# Image Reading and Training

In [25]:
import pandas as pd
import os
from PIL import Image
import numpy as np

# Load CSV
df = pd.read_csv('dataset/Train.csv')

# Select N classes (example: 6 classes)
selected_classes = [0, 1, 2, 3, 4, 5, 6]
subset_df = df[df['ClassId'].isin(selected_classes)]


In [26]:
# Sample 100 images from each class
sampled_df = subset_df.groupby('ClassId').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)
sampled_df

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,29,30,6,6,24,25,0,Train/0/00000_00001_00000.png
1,56,54,5,5,50,49,0,Train/0/00000_00005_00022.png
2,69,70,7,6,63,64,0,Train/0/00000_00002_00024.png
3,60,64,5,6,55,59,0,Train/0/00000_00006_00019.png
4,27,26,6,5,22,21,0,Train/0/00000_00002_00000.png
...,...,...,...,...,...,...,...,...
695,32,33,5,5,27,28,6,Train/6/00006_00000_00005.png
696,39,41,5,6,34,36,6,Train/6/00006_00001_00015.png
697,33,34,6,6,28,29,6,Train/6/00006_00009_00011.png
698,35,34,6,5,29,29,6,Train/6/00006_00013_00012.png


In [27]:
sampled_df.columns

Index(['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId',
       'Path'],
      dtype='object')

Processing Pipeline

In [ ]:
# def process_image(image):
#     # STEP 1: Preprocessing - Apply ONE appropriate filter
#     denoised = median_filter(image)  # You can swap with gaussian_filter or adaptive_median_filter

#     # STEP 2: Convert to HSV
#     hsv = rgb_to_hsv(denoised)

#     # STEP 3: Segment red or blue signs
#     mask = segment_red_blue(hsv)
#     if np.sum(mask) < 50:
#         print(f"Warning: very few foreground pixels in HSV mask at index {idx}")

#     # STEP 4: Post-processing
#     binary = binary_threshold(mask, threshold=127)
#     opened = opening(binary, kernel_size=3)
#     cleaned = connected_component_filtering(opened, area_threshold=300)
#     filled = fill_holes(cleaned)

#     # STEP 5: Edge Detection (Canny)
#     magnitude, direction = sobel_gradients(filled)
#     nms = non_max_suppression(magnitude, direction)
#     dt_result, weak, strong = double_thresholding(nms)
#     edges = edge_tracking_by_hysteresis(dt_result, weak, strong)

#     # STEP 6: Geometric Normalization
#     angle = 0  # Optional: compute based on contours or principal axis
#     rotated = rotate_image(edges, angle_deg=angle)
#     normalized = scale_image(rotated, size=(200, 200))

#     # Return processed image for feature extraction
#     return normalized, hsv, filled


In [ ]:
# import cv2
# import numpy as np
# from tqdm import tqdm

# # Add new columns to store processed data
# sampled_df['Normalized'] = None
# sampled_df['HSV'] = None
# sampled_df['BinaryMask'] = None

# # Processing function applied to all rows
# for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df)):
#     img_path = "dataset/" + row['Path']
#     image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image using OpenCV
    
#     if image is None:
#         print(f"Warning: Couldn't read image at {img_path}")
#         continue

#     # Apply full processing pipeline
#     try:
#         normalized, hsv_img, binary_mask = process_image(image)

#         # Store results back in DataFrame
#         sampled_df.at[idx, 'Normalized'] = normalized
#         sampled_df.at[idx, 'HSV'] = hsv_img
#         sampled_df.at[idx, 'BinaryMask'] = binary_mask

#     except Exception as e:
#         print(f"Error processing image at {img_path}: {e}")


100%|██████████| 700/700 [00:01<00:00, 684.60it/s]


In [42]:
sampled_df

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path,Normalized,HSV,BinaryMask
0,29,30,6,6,24,25,0,Train/0/00000_00001_00000.png,None,None,None
1,56,54,5,5,50,49,0,Train/0/00000_00005_00022.png,None,None,None
2,69,70,7,6,63,64,0,Train/0/00000_00002_00024.png,None,None,None
3,60,64,5,6,55,59,0,Train/0/00000_00006_00019.png,None,None,None
4,27,26,6,5,22,21,0,Train/0/00000_00002_00000.png,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
695,32,33,5,5,27,28,6,Train/6/00006_00000_00005.png,None,None,None
696,39,41,5,6,34,36,6,Train/6/00006_00001_00015.png,None,None,None
697,33,34,6,6,28,29,6,Train/6/00006_00009_00011.png,None,None,None
698,35,34,6,5,29,29,6,Train/6/00006_00013_00012.png,None,None,None


In [119]:
sampled_df = sampled_df.iloc[:15, :]

In [31]:
from tqdm import tqdm


In [120]:
import cv2
import numpy as np
from tqdm import tqdm
import os
from PIL import Image

def process_image(image_path):
    """
    Process an image through the pipeline
    
    Args:
        image_path: Path to the image file
        
    Returns:
        normalized: Normalized processed image
        hsv: HSV version of the image
        filled: Binary mask after processing
    """
    # STEP 1: Read image using PIL
    try:
        # For color processing
        color_image = np.array(Image.open(image_path))
        # For grayscale preprocessing
        gray_image = np.array(Image.open(image_path).convert('L'))
    except Exception as e:
        print(f"Error reading image {image_path}: {e}")
        return None, None, None
    
    # STEP 2: Preprocessing on grayscale image
    # Apply filter to the grayscale image
    denoised_gray = gaussian_filter(gray_image)  # or gaussian_filter or adaptive_median_filter

    filtered_r = gaussian_filter(color_image[:, :, 0])
    filtered_g = gaussian_filter(color_image[:, :, 1])
    filtered_b = gaussian_filter(color_image[:, :, 2])
    filtered_rgb = np.stack((filtered_r, filtered_g, filtered_b), axis=2)
    
    filtered_r = adaptive_median_filter(filtered_rgb[:, :, 0])
    filtered_g = adaptive_median_filter(filtered_rgb[:, :, 1])
    filtered_b = adaptive_median_filter(filtered_rgb[:, :, 2])
    filtered_rgb2 = np.stack((filtered_r, filtered_g, filtered_b), axis=2)
    
    # STEP 3: Convert color image to HSV for color segmentation
    hsv = rgb_to_hsv(filtered_rgb)
    
    # STEP 4: Segment red or blue signs
    # After segment_red_blue
    mask = segment_red_blue(hsv)
    num_foreground = np.sum(mask)
    if num_foreground < 50:
        print(f"[{idx}] Warning: Very few foreground pixels in HSV mask: {num_foreground}")
    else:
        print(f"[{idx}] HSV mask foreground pixels: {num_foreground}")

    # STEP 5: Post-processing on the mask
    binary = binary_threshold(mask, threshold=127)
    opened = opening(binary, kernel_size=3)
    cleaned = connected_component_filtering(opened, area_threshold=50)
    filled = fill_holes(cleaned)

    print(f"[{idx}] Mask pixels before thresholding: {np.sum(mask)}, after thresholding: {np.sum(binary)}")
    print(f"[{idx}] After opening: {np.sum(opened)}, After connected-component filtering: {np.sum(cleaned)}")
    print(f"[{idx}] After filling holes: {np.sum(filled)}")
    
    # STEP 6: Edge Detection (Canny) on the filled binary mask
    magnitude, direction = sobel_gradients(filled)
    nms = non_max_suppression(magnitude, direction)
    dt_result, weak, strong = double_thresholding(nms)
    edges = edge_tracking_by_hysteresis(dt_result, weak, strong)
    
    # STEP 7: Geometric Normalization
    angle = 0  # Optional: compute based on contours or principal axis
    rotated = rotate_image(edges, angle_deg=angle)
    normalized = scale_image(rotated, size=(200, 200))
    
    # Return processed image for feature extraction
    return normalized, hsv, filled

# Prepare DataFrame to store results
sampled_df['processed'] = False  # Flag to track successfully processed images

# Optional: Create directory to save processed images for debugging
output_dir = "processed_images"
os.makedirs(output_dir, exist_ok=True)

# Process all images
print("Processing images...")
for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df)):
    img_path = os.path.join("dataset", row['Path'])
    
    try:
        # Process the image
        normalized, hsv_img, binary_mask = process_image(img_path)
        
        if normalized is None:
            continue
            
        # Optional: Save processed images for debugging
        if idx % 50 == 0:
            filename = os.path.basename(row['Path'])
            # Convert to PIL image and save
            Image.fromarray(normalized).save(os.path.join(output_dir, f"norm_{filename}.png"))
            Image.fromarray(binary_mask).save(os.path.join(output_dir, f"mask_{filename}.png"))
        
        # Mark as successfully processed
        sampled_df.at[idx, 'processed'] = True
        
    except Exception as e:
        print(f"Error processing image at {img_path}: {str(e)}")

print(f"Successfully processed {sampled_df['processed'].sum()} out of {len(sampled_df)} images.")

C:\Users\Mam Hifza Tahir\AppData\Local\Temp\ipykernel_18024\1188267522.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df['processed'] = False  # Flag to track successfully processed images


Processing images...


  0%|          | 0/15 [00:00<?, ?it/s]

[0] HSV mask foreground pixels: 26010


  7%|▋         | 1/15 [00:00<00:03,  3.96it/s]

...

[13] HSV mask foreground pixels: 208080


 93%|█████████▎| 14/15 [00:12<00:01,  1.25s/it]

[13] Mask pixels before thresholding: 208080, after thresholding: 208080
[13] After opening: 198390, After connected-component filtering: 198390
[13] After filling holes: 3894


100%|██████████| 15/15 [00:13<00:00,  1.14it/s]

[14] HSV mask foreground pixels: 130815
[14] Mask pixels before thresholding: 130815, after thresholding: 130815
[14] After opening: 126225, After connected-component filtering: 126225
[14] After filling holes: 1936
Successfully processed 15 out of 15 images.


In [28]:
sampled_df = sampled_df.iloc[:150, :]

In [106]:

def colorSeperator(hsv_img):
    h, s, v = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]
    
    red_mask_low = (h <= 15) & (s >= 100) & (v >= 80)
    red_mask_high = (h >= 165) & (h <= 180) & (s >= 100) & (v >= 80)
    red_mask = red_mask_low | red_mask_high
    
    blue_mask = (h >= 100) & (h <= 130) & (s >= 100) & (v >= 80)
    
    mask = red_mask | blue_mask
    
    if np.sum(red_mask) > np.sum(blue_mask):
        color = 'red'
    else:
        color = 'blue'
    
    return mask, color

def threshold(mask, threshold=0.5):
    return mask > threshold

# def opening(self, img, kernel_size=3):
#     eroded = self.erosion(img, kernel_size)
#     return self.dilation(eroded, kernel_size)

# def closing(self, img, kernel_size=3):
#     dilated = self.dilation(img, kernel_size)
#     return self.erosion(dilated, kernel_size)

def connected_components(binary_img):
    labeled_img = np.zeros_like(binary_img, dtype=int)
    
    current_label = 1
    equiv_table = {}
    
    for i in range(binary_img.shape[0]):
        for j in range(binary_img.shape[1]):
            if binary_img[i, j] == 1:
                neighbors = []
                
                if i > 0 and labeled_img[i-1, j] > 0:
                    neighbors.append(labeled_img[i-1, j])
                
                if j > 0 and labeled_img[i, j-1] > 0:
                    neighbors.append(labeled_img[i, j-1])
                
                if len(neighbors) == 0:
                    labeled_img[i, j] = current_label
                    equiv_table[current_label] = {current_label}
                    current_label += 1
                else:
                    min_label = min(neighbors)
                    labeled_img[i, j] = min_label
                    
                    for n in neighbors:
                        if n != min_label:
                            equiv_table[min_label].update(equiv_table[n])
                            for label in equiv_table[n]:
                                equiv_table[label] = equiv_table[min_label]
    
    relabeled = np.zeros_like(labeled_img)
    new_labels = {}
    current_new_label = 1
    
    for i in range(labeled_img.shape[0]):
        for j in range(labeled_img.shape[1]):
            if labeled_img[i, j] > 0:
                label = labeled_img[i, j]
                min_equiv = min(equiv_table[label])
                
                if min_equiv not in new_labels:
                    new_labels[min_equiv] = current_new_label
                    current_new_label += 1
                
                relabeled[i, j] = new_labels[min_equiv]
    
    return relabeled


def remove_small_components(labeled_img, min_area=100):
    sizes = {}
    for i in range(labeled_img.shape[0]):
        for j in range(labeled_img.shape[1]):
            label = labeled_img[i, j]
            if label > 0:
                if label not in sizes:
                    sizes[label] = 0
                sizes[label] += 1
    
    filtered = np.zeros_like(labeled_img)
    largest_label = None
    largest_size = 0
    
    for label, size in sizes.items():
        if size > largest_size:
            largest_size = size
            largest_label = label
    
    for i in range(labeled_img.shape[0]):
        for j in range(labeled_img.shape[1]):
            if labeled_img[i, j] == largest_label:
                filtered[i, j] = 1
    
    return filtered

def fill_holes(binary_img):
    mask = np.zeros((binary_img.shape[0] + 2, binary_img.shape[1] + 2), dtype=np.uint8)
    
    filled = binary_img.copy()
    mask[1:-1, 1:-1] = 1 - filled
    
    old_mask = np.zeros_like(mask)
    
    mask[0, :] = 1
    mask[-1, :] = 1
    mask[:, 0] = 1
    mask[:, -1] = 1
    
    while not np.array_equal(old_mask, mask):
        old_mask = mask.copy()
        
        for i in range(1, mask.shape[0] - 1):
            for j in range(1, mask.shape[1] - 1):
                if mask[i, j] == 1:
                    if (mask[i-1, j] == 2 or mask[i+1, j] == 2 or 
                        mask[i, j-1] == 2 or mask[i, j+1] == 2):
                        mask[i, j] = 2
        
        mask[0, :] = 1
        mask[-1, :] = 1
        mask[:, 0] = 1
        mask[:, -1] = 1
    
    return 1 - (mask[1:-1, 1:-1] == 1).astype(np.uint8)


In [32]:

def process_image(image_path, idx=None):
    """
    Process an image through the pipeline
    
    Args:
        image_path: Path to the image file
        
    Returns:
        normalized: Normalized processed image
        hsv: HSV version of the image
        filled: Binary mask after processing
    """
    # STEP 1: Read image using PIL
    try:
        # For color processing
        color_image = np.array(Image.open(image_path))
        # For grayscale preprocessing
        gray_image = np.array(Image.open(image_path).convert('L'))
    except Exception as e:
        print(f"Error reading image {image_path}: {e}")
        return None, None, None
    
    # STEP 2: Preprocessing on grayscale image
    # Apply filter to the grayscale image
    denoised_gray = gaussian_filter(gray_image)  # or gaussian_filter or adaptive_median_filter

    filtered_r = gaussian_filter(color_image[:, :, 0])
    filtered_g = gaussian_filter(color_image[:, :, 1])
    filtered_b = gaussian_filter(color_image[:, :, 2])
    filtered_rgb = np.stack((filtered_r, filtered_g, filtered_b), axis=2)
    
    # filtered_r = adaptive_median_filter(filtered_rgb[:, :, 0])
    # filtered_g = adaptive_median_filter(filtered_rgb[:, :, 1])
    # filtered_b = adaptive_median_filter(filtered_rgb[:, :, 2])
    # filtered_rgb2 = np.stack((filtered_r, filtered_g, filtered_b), axis=2)
    
    # STEP 3: Convert color image to HSV for color segmentation
    hsv = rgb_to_hsv(filtered_rgb)
    
    # STEP 4: Segment red or blue signs
    # After segment_red_blue
    mask = segment_red_blue(hsv)
    num_foreground = np.sum(mask)
    if num_foreground < 50:
        print(f"[{idx}] Warning: Very few foreground pixels in HSV mask: {num_foreground}")
    else:
        print(f"[{idx}] HSV mask foreground pixels: {num_foreground}")

    # STEP 5: Post-processing on the mask
    binary = binary_threshold(mask, threshold=20)
    opened = opening(binary, kernel_size=3)
    cleaned = connected_component_filtering(opened, area_threshold=20)
    filled = fill_holes(cleaned)

    print(f"[{idx}] Mask mean: {np.mean(mask)}, min: {np.min(binary)}, max: {np.max(binary)}")
    print(f"[{idx}] Mask pixels before thresholding: {np.sum(mask)}, after thresholding: {np.sum(binary)}")
    print(f"[{idx}] After opening: {np.sum(opened)}, After connected-component filtering: {np.sum(cleaned)}")
    print(f"[{idx}] After filling holes: {np.sum(filled)}")
    
    # STEP 6: Edge Detection (Canny) on the filled binary mask
    magnitude, direction = sobel_gradients(filled)
    nms = non_max_suppression(magnitude, direction)
    dt_result, weak, strong = double_thresholding(nms)
    edges = edge_tracking_by_hysteresis(dt_result, weak, strong)
    
    # STEP 7: Geometric Normalization
    angle = 0  # Optional: compute based on contours or principal axis
    rotated = rotate_image(edges, angle_deg=angle)
    normalized = scale_image(rotated, size=(200, 200))
    
    # Return processed image for feature extraction
    return normalized, hsv, filled

def extract_features_batch():
    """
    Extract features from all processed images
    
    Returns:
        DataFrame with features added
    """
    # Create columns for features
    feature_columns = ['corner_count', 'circularity', 'aspect_ratio', 'extent', 'avg_hue']
    for col in feature_columns:
        sampled_df[col] = None
    
    print("Extracting features...")
    for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df)):
        # if not row['processed']:
        #     continue
            
        img_path = os.path.join("dataset", row['Path'])
        
        try:
            # Process the image again
            normalized, hsv_img, binary_mask = process_image(img_path, idx=idx)

            if np.sum(binary_mask) == 0:
                print(f"[{idx}] Skipping: binary_mask is empty after processing: {img_path}")
                continue

            if np.max(normalized) == 0:
                print(f"[{idx}] Warning: normalized image is completely black: {img_path}")
            
            if normalized is None:
                continue
                
            # Debug dimensions
            if idx % 100 == 0:
                print(f"Debug - Image {idx}:")
                print(f"  normalized shape: {normalized.shape}")
                print(f"  hsv_img shape: {hsv_img.shape}")
                print(f"  binary_mask shape: {binary_mask.shape}")
            
            # Check and fix dimensions
            # If normalized is grayscale, convert to 3D array
            if len(normalized.shape) == 2:
                normalized = np.stack([normalized, normalized, normalized], axis=2)
                
            # Extract features with dimension-safe version
            features = extract_features_safe(normalized, binary_mask, hsv_img)
            
            # Store features in DataFrame
            for feature, value in features.items():
                sampled_df.at[idx, feature] = value
                
        except Exception as e:
            print(f"Error extracting features from {img_path}: {str(e)}")
            # Print more debug info for errors
            try:
                print(f"  normalized shape: {normalized.shape if normalized is not None else 'None'}")
                print(f"  hsv_img shape: {hsv_img.shape if hsv_img is not None else 'None'}")
                print(f"  binary_mask shape: {binary_mask.shape if binary_mask is not None else 'None'}")
            except:
                print("  Could not print debug info")
    
    print(f"Successfully extracted features from {sampled_df[feature_columns[0]].notna().sum()} images.")
    return sampled_df

def extract_features_safe(normalized_img, binary_mask, hsv_img):
    """
    Safe version of extract_features that handles different dimension inputs
    """
    # Check dimensions and fix if needed
    if normalized_img is None or binary_mask is None or hsv_img is None:
        return {
            "corner_count": 0,
            "circularity": 0,
            "aspect_ratio": 0,
            "extent": 0,
            "avg_hue": 0
        }
    
    # Ensure binary_mask is binary
    binary = (binary_mask > 0).astype(np.uint8)
    
    # Convert normalized_img to RGB if it's grayscale
    if len(normalized_img.shape) == 2:
        # Single channel - convert to 3 channels
        normalized_rgb = np.stack([normalized_img, normalized_img, normalized_img], axis=2)
    else:
        normalized_rgb = normalized_img
    
    # Calculate grayscale for corner detection
    if len(normalized_rgb.shape) == 3 and normalized_rgb.shape[2] >= 3:
        gray = np.mean(normalized_rgb[:,:,:3], axis=2).astype(np.float32)
    else:
        gray = normalized_rgb.astype(np.float32)
        
    # 1. Corner Count
    try:
        corners = harris_corner_count(gray)
    except Exception as e:
        print(f"Error in corner detection: {e}")
        corners = 0
        
    # 2. Circularity
    area = np.sum(binary)
    try:
        perimeter = compute_perimeter(binary)
        circularity = (4 * np.pi * area) / (perimeter**2) if perimeter > 0 else 0
    except Exception as e:
        print(f"Error in circularity calculation: {e}")
        circularity = 0
        
    # 3. Bounding box and Aspect Ratio
    ys, xs = np.where(binary == 1)
    if len(xs) == 0 or len(ys) == 0:
        aspect_ratio = 0
        extent = 0
    else:
        x_min, x_max = xs.min(), xs.max()
        y_min, y_max = ys.min(), ys.max()
        width = x_max - x_min + 1
        height = y_max - y_min + 1
        aspect_ratio = width / height if height > 0 else 0
        
        # 4. Extent
        bbox_area = width * height
        extent = area / bbox_area if bbox_area > 0 else 0
        
    # 5. Average Hue - handle different HSV formats
    try:
        if len(hsv_img.shape) == 3 and hsv_img.shape[2] >= 3:
            # Standard HSV format (H,W,3)
            hue = hsv_img[:,:,0]
        elif len(hsv_img.shape) == 3 and hsv_img.shape[0] == 3:
            # Channel-first format (3,H,W)
            hue = hsv_img[0,:,:]
        elif len(hsv_img.shape) == 2:
            # Already hue channel
            hue = hsv_img
        else:
            hue = np.zeros_like(binary, dtype=float)
            
        # Calculate average hue where mask is active
        if np.any(binary):
            avg_hue = np.mean(hue[binary == 1])
        else:
            avg_hue = 0
    except Exception as e:
        print(f"Error in hue calculation: {e}")
        avg_hue = 0
        
    return {
        "corner_count": int(corners),
        "circularity": float(circularity),
        "aspect_ratio": float(aspect_ratio),
        "extent": float(extent),
        "avg_hue": float(avg_hue)
    }

In [33]:
extract_features_batch()

C:\Users\Mam Hifza Tahir\AppData\Local\Temp\ipykernel_19624\960625423.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df[col] = None


Extracting features...


  0%|          | 0/150 [00:00<?, ?it/s]

[0] HSV mask foreground pixels: 26010
[0] Mask mean: 29.896551724137932, min: 0, max: 255
[0] Mask pixels before thresholding: 26010, after thresholding: 26010
[0] After opening: 12495, After connected-component filtering: 12495
[0] After filling holes: 12495
Debug - Image 0:
  normalized shape: (200, 200)
  hsv_img shape: (30, 29, 3)
  binary_mask shape: (30, 29)


  1%|          | 1/150 [00:01<04:18,  1.73s/it]

...

 99%|█████████▉| 149/150 [04:04<00:01,  1.26s/it]

[149] HSV mask foreground pixels: 49470
[149] Mask mean: 28.04421768707483, min: 0, max: 255
[149] Mask pixels before thresholding: 49470, after thresholding: 49470
[149] After opening: 26010, After connected-component filtering: 22440
[149] After filling holes: 22440


100%|██████████| 150/150 [04:06<00:00,  1.64s/it]

Successfully extracted features from 105 images.


,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path,corner_count,circularity,aspect_ratio,extent,avg_hue
0,29,30,6,6,24,25,0,Train/0/00000_00001_00000.png,322,0.304075,1.571429,0.636364,51.285714
1,56,54,5,5,50,49,0,Train/0/00000_00005_00022.png,1773,0.010291,1.152174,0.542658,45.909297
2,69,70,7,6,63,64,0,Train/0/00000_00002_00024.png,2646,0.019852,1.04,0.243462,6.301738
3,60,64,5,6,55,59,0,Train/0/00000_00006_00019.png,1777,0.006901,0.886792,0.735849,22.301146
4,27,26,6,5,22,21,0,Train/0/00000_00002_00000.png,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,114,118,10,11,104,108,1,Train/1/00001_00000_00029.png,None,None,None,None,None
146,29,32,6,5,23,27,1,Train/1/00001_00028_00001.png,None,None,None,None,None
147,36,35,5,5,31,30,1,Train/1/00001_00039_00003.png,None,None,None,None,None
148,94,122,8,11,91,112,1,Train/1/00001_00053_00029.png,1765,0.002247,0.797872,0.793191,12.177396


In [128]:
sampled_df.columns

Index(['ClassId', 'Path', 'processed', 'corner_count', 'circularity',
       'aspect_ratio', 'extent', 'avg_hue'],
      dtype='object')

In [121]:
sampled_df = sampled_df.drop(['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2'], axis=1)


In [89]:
# sampled_df['avg_hue'].value_counts()
sampled_df['avg_hue'].isna().sum()

5

In [73]:
sampled_df['extent'].value_counts()

extent
0.17199612403100775    1
0.3055783429040197     1
0.178                  1
0.7378606615059817     1
0.29835507657402155    1
0.17706131078224102    1
0.2783400809716599     1
0.38492063492063494    1
0.2833333333333333     1
0.29005847953216374    1
0.2613981762917933     1
0.2730364873222016     1
0.3011204481792717     1
0.2206439393939394     1
0.7800925925925926     1
0.7791452991452992     1
0.18640148011100832    1
0.3888888888888889     1
0.28511627906976744    1
0.3037593984962406     1
0.7901543596161869     1
0.29830210772833726    1
0.22548028311425683    1
0.5510204081632653     1
0.6                    1
0.46524064171123       1
Name: count, dtype: int64

In [38]:
sampled_df['aspect_ratio'].value_counts()

aspect_ratio
0.0                   37
0.8679245283018868     1
0.6086956521739131     1
0.9180327868852459     1
0.9215686274509803     1
1.0                    1
0.9615384615384616     1
0.9264705882352942     1
1.103896103896104      1
0.4791666666666667     1
0.9122807017543859     1
0.9534883720930233     1
0.8448275862068966     1
0.5409836065573771     1
Name: count, dtype: int64

In [86]:
# Debug Steps and Solutions for Feature Extraction Issues

## Step 1: Add Debug Print Statements to Identify Failures

# First, let's add better debug prints to identify where the pipeline is failing:

# ```python
def process_image(image_path, debug=False, idx=None):
    """
    Process an image through the pipeline with debug options
    
    Args:
        image_path: Path to the image file
        debug: Whether to print debug information
        idx: Index for logging purposes
        
    Returns:
        normalized: Normalized processed image
        hsv: HSV version of the image
        filled: Binary mask after processing
    """
    if debug:
        print(f"\nProcessing image {idx}: {image_path}")
    
    # STEP 1: Read image using PIL
    try:
        # For color processing
        color_image = np.array(Image.open(image_path))
        # For grayscale preprocessing
        gray_image = np.array(Image.open(image_path).convert('L'))
        
        if debug:
            print(f"  Image loaded: color shape={color_image.shape}, gray shape={gray_image.shape}")
    except Exception as e:
        print(f"Error reading image {image_path}: {e}")
        return None, None, None
    
    # STEP 2: Preprocessing on grayscale image
    try:
        # Apply filter to the grayscale image
        denoised_gray = median_filter(gray_image)  # or gaussian_filter or adaptive_median_filter
        if debug:
            print(f"  Preprocessing: denoised shape={denoised_gray.shape}")
    except Exception as e:
        print(f"Error in preprocessing: {e}")
        return None, None, None
    
    # STEP 3: Convert color image to HSV for color segmentation
    try:
        hsv = rgb_to_hsv(color_image)
        if debug:
            print(f"  HSV conversion: hsv shape={hsv.shape}")
    except Exception as e:
        print(f"Error in HSV conversion: {e}")
        return None, None, None
    
    # STEP 4: Segment red or blue signs
    try:
        mask = segment_red_blue(hsv)
        if debug:
            print(f"  Segmentation: mask shape={mask.shape}, sum={np.sum(mask)}")
        
        if np.sum(mask) < 50:
            print(f"Warning: very few foreground pixels in HSV mask: {np.sum(mask)} pixels")
            # Don't return None yet, try to continue processing
    except Exception as e:
        print(f"Error in segmentation: {e}")
        return None, None, None
    
    # STEP 5: Post-processing on the mask
    try:
        binary = binary_threshold(mask, threshold=127)
        opened = opening(binary, kernel_size=3)
        cleaned = connected_component_filtering(opened, area_threshold=100)  # Reduced threshold
        filled = fill_holes(cleaned)
        
        if debug:
            print(f"  Post-processing: binary sum={np.sum(binary)}, opened sum={np.sum(opened)}, "
                  f"cleaned sum={np.sum(cleaned)}, filled sum={np.sum(filled)}")
        
        if np.sum(filled) < 10:  # Very minimal check
            print(f"Warning: very few pixels in final mask: {np.sum(filled)} pixels")
            # Return early if mask is essentially empty
            if np.sum(filled) == 0:
                return None, hsv, filled
    except Exception as e:
        print(f"Error in post-processing: {e}")
        return None, None, None
    
    # STEP 6: Edge Detection (Canny) on the filled binary mask
    try:
        magnitude, direction = sobel_gradients(filled)
        nms = non_max_suppression(magnitude, direction)
        dt_result, weak, strong = double_thresholding(nms)
        edges = edge_tracking_by_hysteresis(dt_result, weak, strong)
        
        if debug:
            print(f"  Edge detection: edges sum={np.sum(edges)}")
    except Exception as e:
        print(f"Error in edge detection: {e}")
        return None, hsv, filled  # Return what we have so far
    
    # STEP 7: Geometric Normalization
    try:
        angle = 0  # Optional: compute based on contours or principal axis
        rotated = rotate_image(edges, angle_deg=angle)
        normalized = scale_image(rotated, size=(200, 200))
        
        if debug:
            print(f"  Normalization: normalized shape={normalized.shape}")
    except Exception as e:
        print(f"Error in normalization: {e}")
        return None, hsv, filled
    
    # Return processed image for feature extraction
    return normalized, hsv, filled
# ```

## Step 2: Fix HSV Segmentation Function

# Your segmentation function might be too strict. Let's modify it:

# ```python
def segment_red_blue(hsv_img):
    """
    Segment red and blue signs based on HSV values
    
    Args:
        hsv_img: HSV image
        
    Returns:
        mask: Binary mask where signs are detected
    """
    # Check if H is first channel (some implementations swap order)
    h_channel = hsv_img[:,:,0]
    s_channel = hsv_img[:,:,1]
    v_channel = hsv_img[:,:,2]
    
    # Verify the range of hue values (0-180 or 0-360)
    h_max = np.max(h_channel)
    h_scale = 180 if h_max <= 180 else 360
    
    # Scale factors for normalization
    s_scale = 255 if np.max(s_channel) > 1 else 1
    v_scale = 255 if np.max(v_channel) > 1 else 1
    
    # Adjust thresholds based on scale
    if h_scale == 360:
        red_low1, red_high1 = 0, 30
        red_low2, red_high2 = 330, 360
        blue_low, blue_high = 200, 260
    else:  # 180 scale
        red_low1, red_high1 = 0, 15
        red_low2, red_high2 = 165, 180
        blue_low, blue_high = 100, 130
    
    # Normalize saturation and value thresholds
    s_thresh = 50 / s_scale  # More permissive than 100
    v_thresh = 50 / v_scale  # More permissive than 80
    
    # Create masks with more permissive thresholds
    red_mask1 = (h_channel >= red_low1) & (h_channel <= red_high1) & (s_channel >= s_thresh) & (v_channel >= v_thresh)
    red_mask2 = (h_channel >= red_low2) & (h_channel <= red_high2) & (s_channel >= s_thresh) & (v_channel >= v_thresh)
    blue_mask = (h_channel >= blue_low) & (h_channel <= blue_high) & (s_channel >= s_thresh) & (v_channel >= v_thresh)
    
    # Combine masks
    combined_mask = red_mask1 | red_mask2 | blue_mask
    
    return combined_mask.astype(np.uint8) * 255
# ```

## Step 3: Fix Connected Component Filtering

# Your connected component filtering might be too aggressive. Let's modify it:

# ```python
def label_connected_components(binary_image):
    h, w = binary_image.shape
    labels = np.zeros((h, w), dtype=np.int32)
    label = 1

    for i in range(h):
        for j in range(w):
            if binary_image[i, j] == 255 and labels[i, j] == 0:
                # Start new component
                stack = [(i, j)]
                while stack:
                    x, y = stack.pop()
                    if 0 <= x < h and 0 <= y < w:
                        if binary_image[x, y] == 255 and labels[x, y] == 0:
                            labels[x, y] = label
                            stack.extend([(x+1, y), (x-1, y), (x, y+1), (x, y-1)])
                label += 1

    return labels, label - 1  # label - 1 = number of components


def connected_component_filtering(binary_img, area_threshold=100):
    """
    Filter small connected components
    
    Args:
        binary_img: Binary image
        area_threshold: Minimum area to keep
        
    Returns:
        filtered: Filtered binary image
    """
    # Create output image
    filtered = np.zeros_like(binary_img)
    
    # Find connected components
    labeled, num_features = label_connected_components(binary_img)
    
    if num_features == 0:
        return binary_img  # Return original if no components found
    
    # Count pixels in each component
    component_sizes = np.zeros(num_features + 1, dtype=int)
    for i in range(binary_img.shape[0]):
        for j in range(binary_img.shape[1]):
            if labeled[i, j] > 0:
                component_sizes[labeled[i, j]] += 1
    
    # Keep only components above threshold
    for i in range(binary_img.shape[0]):
        for j in range(binary_img.shape[1]):
            if labeled[i, j] > 0 and component_sizes[labeled[i, j]] >= area_threshold:
                filtered[i, j] = 255
    
    if np.sum(filtered) == 0:
        # If no components remain, keep the largest component
        largest_label = np.argmax(component_sizes)
        if largest_label > 0:  # Make sure it's not background
            for i in range(binary_img.shape[0]):
                for j in range(binary_img.shape[1]):
                    if labeled[i, j] == largest_label:
                        filtered[i, j] = 255
    
    return filtered
# ```

## Step 4: Fix Feature Extraction Pipeline

# Let's ensure our extract_features function can handle various cases:

# ```python
def extract_features(normalized_img, binary_mask, hsv_img):
    """
    Extract features from processed images
    
    Args:
        normalized_img: Normalized image
        binary_mask: Binary mask of the sign
        hsv_img: HSV version of the image
        
    Returns:
        dict: Dictionary of features
    """
    # Handle None inputs
    if normalized_img is None or binary_mask is None or hsv_img is None:
        return {
            "corner_count": 0,
            "circularity": 0,
            "aspect_ratio": 0,
            "extent": 0,
            "avg_hue": 0
        }
    
    # Ensure binary_mask is binary
    binary = (binary_mask > 0).astype(np.uint8)
    
    # If no foreground pixels, return zeros
    if np.sum(binary) == 0:
        return {
            "corner_count": 0,
            "circularity": 0,
            "aspect_ratio": 0,
            "extent": 0,
            "avg_hue": 0
        }
    
    # Convert normalized_img to grayscale if needed
    if len(normalized_img.shape) == 3:
        gray = np.mean(normalized_img, axis=2).astype(np.float32)
    else:
        gray = normalized_img.astype(np.float32)
    
    # 1. Corner Count
    corners = harris_corner_detector(gray)
    corner_count = np.sum(corners > 0)
    
    # 2. Circularity
    area = np.sum(binary)
    perimeter = calculate_perimeter(binary)
    circularity = (4 * np.pi * area) / (perimeter**2) if perimeter > 0 else 0
    
    # 3. Aspect Ratio
    rows, cols = np.where(binary > 0)
    if len(rows) > 0 and len(cols) > 0:
        min_row, max_row = np.min(rows), np.max(rows)
        min_col, max_col = np.min(cols), np.max(cols)
        height = max_row - min_row + 1
        width = max_col - min_col + 1
        aspect_ratio = width / height if height > 0 else 0
        
        # 4. Extent
        bbox_area = width * height
        extent = area / bbox_area if bbox_area > 0 else 0
    else:
        aspect_ratio = 0
        extent = 0
    
    # 5. Average Hue
    if len(hsv_img.shape) == 3:
        hue_channel = hsv_img[:,:,0]
    else:
        hue_channel = hsv_img
    
    # Calculate average hue where mask is active
    hue_values = hue_channel[binary > 0]
    avg_hue = np.mean(hue_values) if len(hue_values) > 0 else 0
    
    return {
        "corner_count": int(corner_count),
        "circularity": float(circularity),
        "aspect_ratio": float(aspect_ratio),
        "extent": float(extent),
        "avg_hue": float(avg_hue)
    }
# ```

## Step 5: Update the Extract Features Batch Function
# 
# Let's improve the batch processing function:

# ```python
def extract_features_batch(sampled_df):
    """
    Extract features from all processed images
    
    Args:
        sampled_df: DataFrame with image paths
        
    Returns:
        DataFrame with features added
    """
    # Create columns for features
    feature_columns = ['corner_count', 'circularity', 'aspect_ratio', 'extent', 'avg_hue']
    for col in feature_columns:
        sampled_df[col] = None
    
    # Add debug columns
    debug_columns = ['has_mask', 'mask_size', 'processing_status']
    for col in debug_columns:
        sampled_df[col] = None
    
    print("Extracting features...")
    success_count = 0
    for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df)):
        img_path = os.path.join("dataset", row['Path'])
        
        try:
            # Process the image
            normalized, hsv_img, binary_mask = process_image(img_path, debug=(idx % 100 == 0), idx=idx)
            
            # Save debug info
            sampled_df.at[idx, 'has_mask'] = binary_mask is not None
            sampled_df.at[idx, 'mask_size'] = np.sum(binary_mask) if binary_mask is not None else 0
            
            # Extract features
            features = extract_features(normalized, binary_mask, hsv_img)
            
            # Store features in DataFrame
            for feature, value in features.items():
                sampled_df.at[idx, feature] = value
            
            # Set processing status
            if any(features.values()):  # If any feature is non-zero
                sampled_df.at[idx, 'processing_status'] = 'success'
                success_count += 1
            else:
                sampled_df.at[idx, 'processing_status'] = 'failed'
                
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            sampled_df.at[idx, 'processing_status'] = 'error'
    
    print(f"Successfully extracted features from {success_count} out of {len(sampled_df)} images.")
    
    # Print some statistics
    print("\nFeature Statistics:")
    for col in feature_columns:
        non_zero = (sampled_df[col] > 0).sum()
        print(f"  {col}: {non_zero} non-zero values ({non_zero/len(sampled_df)*100:.1f}%)")
    
    return sampled_df
# ```

## Step 6: Add Visual Debugging for Failed Images

# Let's create a function to visually inspect failed images:

# ```python
def debug_failed_images(sampled_df, output_dir="debug_images", num_samples=10):
    """
    Save debug images for failed feature extraction
    
    Args:
        sampled_df: DataFrame with processing results
        output_dir: Directory to save debug images
        num_samples: Number of failed images to debug
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Get failed images
    failed_df = sampled_df[sampled_df['processing_status'] == 'failed']
    if len(failed_df) == 0:
        print("No failed images to debug.")
        return
    
    # Sample some failed images
    sample_size = min(num_samples, len(failed_df))
    sample_indices = np.random.choice(failed_df.index, size=sample_size, replace=False)
    
    print(f"Debugging {sample_size} failed images...")
    for idx in sample_indices:
        row = sampled_df.loc[idx]
        img_path = os.path.join("dataset", row['Path'])
        
        # Process with debug flag
        normalized, hsv_img, binary_mask = process_image(img_path, debug=True, idx=idx)
        
        # Save original image
        original = np.array(Image.open(img_path))
        Image.fromarray(original).save(os.path.join(output_dir, f"{idx}_1_original.png"))
        
        # Save HSV image (H channel)
        if hsv_img is not None:
            h_channel = hsv_img[:,:,0]
            # Normalize for visualization
            h_vis = (h_channel * 255 / np.max(h_channel)).astype(np.uint8) if np.max(h_channel) > 0 else np.zeros_like(h_channel, dtype=np.uint8)
            Image.fromarray(h_vis).save(os.path.join(output_dir, f"{idx}_2_hue.png"))
        
        # Save binary mask
        if binary_mask is not None:
            Image.fromarray(binary_mask).save(os.path.join(output_dir, f"{idx}_3_mask.png"))
        
        # Save normalized image
        if normalized is not None:
            if len(normalized.shape) == 2:
                Image.fromarray(normalized).save(os.path.join(output_dir, f"{idx}_4_normalized.png"))
            else:
                Image.fromarray(normalized.astype(np.uint8)).save(os.path.join(output_dir, f"{idx}_4_normalized.png"))
        
        print(f"Saved debug images for image {idx}")
# ```

## Step 7: Main Processing Function

# Let's update the main processing function:

# ```python
def main():
    # Load dataset
    # ... (your code to load dataset)
    
    # Process all images
    print("Processing images and extracting features...")
    temp = extract_features_batch(sampled_df)
    
    # Debug failed images
    debug_failed_images(temp)
    
    # Save results
    temp.to_csv("processed_features.csv", index=False)
    print("Results saved to processed_features.csv")
    
    # Print summary
    success_rate = temp[temp['processing_status'] == 'success'].shape[0] / len(temp)
    print(f"Overall success rate: {success_rate*100:.1f}%")
    
    return temp

main()
# ```

## Additional Tips for Successful Feature Extraction

# 1. **Check your filters implementation:**
#    - Make sure your median_filter, gaussian_filter, and other filters are implemented correctly
#    - Verify they return images of the same size as the input

# 2. **HSV color space conversion:**
#    - Double-check your rgb_to_hsv function
#    - Make sure hue is in the expected range (0-180 or 0-360)
#    - Ensure saturation and value are properly normalized

# 3. **Test intermediate results:**
#    - Add code to save intermediate images from each step
#    - Visually inspect these to find where the pipeline is failing

# 4. **Reduce thresholds:**
#    - Be more permissive with thresholds in the segment_red_blue function
#    - Lower the area_threshold in connected_component_filtering

# 5. **Make the pipeline more robust:**
#    - Add more error handling
#    - Use fallback methods when primary methods fail
#    - Add more checks for edge cases (empty images, etc.)

C:\Users\Mam Hifza Tahir\AppData\Local\Temp\ipykernel_12296\1706056554.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df[col] = None
C:\Users\Mam Hifza Tahir\AppData\Local\Temp\ipykernel_12296\1706056554.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df[col] = None
C:\Users\Mam Hifza Tahir\AppData\Local\Temp\ipykernel_12296\1706056554.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

Processing images and extracting features...
Extracting features...


  0%|          | 0/50 [00:00<?, ?it/s]


Processing image 0: dataset\Train/0/00000_00001_00000.png
  Image loaded: color shape=(30, 29, 3), gray shape=(30, 29)
  Preprocessing: denoised shape=(30, 29)
  HSV conversion: hsv shape=(30, 29, 3)
  Segmentation: mask shape=(30, 29), sum=0
  Post-processing: binary sum=0, opened sum=0, cleaned sum=0, filled sum=0


  2%|▏         | 1/50 [00:00<00:09,  5.14it/s]

...

 96%|█████████▌| 48/50 [00:14<00:00,  3.22it/s]

100%|██████████| 50/50 [00:14<00:00,  3.40it/s]

Successfully extracted features from 0 out of 50 images.

Feature Statistics:
  corner_count: 0 non-zero values (0.0%)
  circularity: 0 non-zero values (0.0%)
  aspect_ratio: 0 non-zero values (0.0%)
  extent: 0 non-zero values (0.0%)
  avg_hue: 0 non-zero values (0.0%)
Debugging 10 failed images...

Processing image 20: dataset\Train/0/00000_00002_00015.png
  Image loaded: color shape=(41, 41, 3), gray shape=(41, 41)


  Preprocessing: denoised shape=(41, 41)
  HSV conversion: hsv shape=(41, 41, 3)
  Segmentation: mask shape=(41, 41), sum=0
  Post-processing: binary sum=0, opened sum=0, cleaned sum=0, filled sum=0
Saved debug images for image 20

Processing image 9: dataset\Train/0/00000_00006_00016.png
  Image loaded: color shape=(56, 54, 3), gray shape=(56, 54)
  Preprocessing: denoised shape=(56, 54)
  HSV conversion: hsv shape=(56, 54, 3)
  Segmentation: mask shape=(56, 54), sum=0
  Post-processing: binary sum=0, opened sum=0, cleaned sum=0, filled sum=0
Saved debug images for image 9

Processing image 23: dataset\Train/0/00000_00003_00019.png
  Image loaded: color shape=(50, 50, 3), gray shape=(50, 50)
  Preprocessing: denoised shape=(50, 50)
  HSV conversion: hsv shape=(50, 50, 3)
  Segmentation: mask shape=(50, 50), sum=0
  Post-processing: binary sum=0, opened sum=0, cleaned sum=0, filled sum=0
Saved debug images for image 23

Processing image 22: dataset\Train/0/00000_00004_00027.png
  Image

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path,corner_count,circularity,aspect_ratio,extent,avg_hue,has_mask,mask_size,processing_status
0,29,30,6,6,24,25,0,Train/0/00000_00001_00000.png,0,0,0,0,0,True,0,failed
1,56,54,5,5,50,49,0,Train/0/00000_00005_00022.png,0,0,0,0,0,True,0,failed
2,69,70,7,6,63,64,0,Train/0/00000_00002_00024.png,0,0,0,0,0,True,0,failed
3,60,64,5,6,55,59,0,Train/0/00000_00006_00019.png,0,0,0,0,0,True,0,failed
4,27,26,6,5,22,21,0,Train/0/00000_00002_00000.png,0,0,0,0,0,True,0,failed
5,31,30,5,6,26,25,0,Train/0/00000_00005_00005.png,0,0,0,0,0,True,0,failed
6,38,38,5,6,32,33,0,Train/0/00000_00001_00015.png,0,0,0,0,0,True,0,failed
7,35,38,5,6,30,33,0,Train/0/00000_00006_00002.png,0,0,0,0,0,True,0,failed
8,36,36,6,5,30,30,0,Train/0/00000_00000_00009.png,0,0,0,0,0,True,0,failed
9,54,56,5,5,48,50,0,Train/0/00000_00006_00016.png,0,0,0,0,0,True,0,failed


In [85]:
sampled_df = sampled_df.iloc[:50, :]
sampled_df.shape

(50, 8)